In [1]:
import pandas as pd
import stockdata as sd

In [2]:
df = sd.getdata("select indexname, symbol, 1 x from nseindices")
df = pd.pivot_table(df, index='symbol', columns='indexname')
df.columns = df.columns.droplevel()
df.reset_index(inplace=True)
df.fillna(0, inplace=True)
cols = df.columns[df.dtypes.eq('float64')]
df[cols] = df[cols].apply(pd.to_numeric, errors='ignore', axis=1, downcast='integer')

In [3]:
df.head()

indexname,symbol,NIFTY Aditya Birla Group,NIFTY Mahindra Group,NIFTY SME Emerge,NIFTY Tata Group,NIFTY Tata Group 25% Cap,Nifty 100,Nifty 100 Alpha 30,Nifty 100 Equal Weight,Nifty 100 Low Volatility 30,...,Nifty PSU Bank,Nifty Pharma,Nifty Private Bank,Nifty Quality Low Volatility 30,Nifty Realty,Nifty Services Sector,Nifty Smallcap 100,Nifty Smallcap 250,Nifty Smallcap 50,Nifty100 Quality 30
0,3MINDIA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AAKASH,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AARON,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AARTIIND,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AARVI,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df.to_csv('index_symbols.csv', index=False)